In [1]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import pickle

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

#load pkl in a list
def load_corpus(pkl_path):
    rfc_dict = pickle.load(open( pkl_path, "rb" ))
    return list(rfc_dict.values())

# pkl is saved as a list of strings
cleaned_rfcs = load_corpus('callflow_pkl.pkl')
data_words = []
for s in cleaned_rfcs:
    data_words.append(s.split(' '))
# print(data_words)

In [2]:
bigram = gensim.models.phrases.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.phrases.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example

# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/home/prakhar/anaconda3/envs/onr/lib/python3.5/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [3]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(type(data_lemmatized))
print(data_lemmatized[0])

<class 'list'>
['exampl', 'suppos', 'host', 'receiv', 'router', 'advertis', 'router', 'router', 'lifetim_second', 'default', 'router', 'prefer', 'medium', 'bodi', 'router', 'advertis', 'contain', 'rout', 'inform', 'option', 'rout', 'lifetim_second', 'rout', 'prefer', 'low', 'process', 'router', 'advertis', 'type', 'host', 'router', 'default', 'router', 'list', 'lifetim_second', 'type', 'host', 'receiv', 'router', 'advertis', 'entri', 'router', 'default', 'router', 'list', 'medium', 'prefer', 'lifetim_second', 'type', 'host', 'entri', 'rout', 'tabl', 'router', 'low', 'prefer', 'lifetim_second', 'process', 'router', 'advertis', 'type', 'host', 'may', 'transient', 'state', 'rout', 'tabl', 'router', 'medium', 'prefer', 'lifetim_second']


In [4]:
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print('corpus type: ', type(corpus))

corpus type:  <class 'list'>


In [5]:
num_topics = 40
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=100, 
update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
# coherence and perplexity : metrics for topic model

[(31,
  '0.081*"xro" + 0.039*"int" + 0.033*"string_xro" + 0.030*"ssrc" + '
  '0.021*"unicod" + 0.019*"begin" + 0.015*"normal" + 0.015*"bitcount" + '
  '0.015*"dft" + 0.015*"bitbuf"'),
 (11,
  '0.075*"radiu" + 0.068*"repli" + 0.042*"supportedmethod" + 0.034*"dhcpv" + '
  '0.028*"ge" + 0.025*"otherwis" + 0.025*"server" + 0.020*"fal" + '
  '0.020*"deleg" + 0.012*"solicit"'),
 (29,
  '0.075*"auth" + 0.067*"historyinfo" + 0.064*"hmacalgo" + 0.064*"chunk" + '
  '0.048*"select" + 0.032*"initrandom" + 0.032*"cookieack" + '
  '0.032*"initackrandom" + 0.032*"cookieecho" + 0.028*"idi"'),
 (15,
  '0.069*"offset" + 0.053*"bind" + 0.047*"cbname" + 0.046*"gscbflag" + '
  '0.046*"gs" + 0.046*"gsauthzid" + 0.046*"gsnonstdflag" + 0.046*"saslnam" + '
  '0.046*"gsheader" + 0.026*"xid"'),
 (22,
  '0.098*"sourc" + 0.078*"xselement" + 0.059*"hash" + 0.056*"xscomplextyp" + '
  '0.045*"minoccur" + 0.040*"xsexten" + 0.032*"xscomplexcont" + '
  '0.032*"xssequenc" + 0.020*"xml_version" + 0.017*"encodingutf"'),
 (

In [6]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -7.6388953032837215

Coherence Score:  0.48971456019746384


In [7]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/prakhar/anaconda3/envs/onr/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
16     4.054759        1       1  0.022734 -0.022279
1      3.992245        1       2  0.025276  0.040600
6      3.851135        1       3 -0.005826 -0.093674
21     3.614511        1       4  0.014168  0.028415
17     3.402365        1       5  0.026764 -0.178264
7      3.396220        1       6  0.024368  0.044331
26     3.295944        1       7 -0.307298  0.005993
13     3.052028        1       8  0.032413 -0.296671
36     3.017912        1       9 -0.012366 -0.052980
25     2.980636        1      10  0.025733  0.034592
34     2.847640        1      11  0.034261 -0.102741
8      2.789930        1      12 -0.339434 -0.039778
23     2.720529        1      13  0.025486  0.031307
0      2.713422        1      14  0.030765  0.033118
35     2.705898        1      15  0.032407 -0.095842
18     2.680213        1      16  0.029530  0.042834
27     2.630987        1      17  0.024849  0.049876
20     2.622968        1      18  0.022111  0.017773
32     2.553043        1      19  0.020810  0.010157
24     2.552773        1      20  0.030111  0.023470
30     2.499703        1      21  0.029450 -0.028830
19     2.422886        1      22  0.026104  0.018769
39     2.366158        1      23  0.022717 -0.000062
9      2.342486        1      24  0.027543  0.039837
4      2.320288        1      25  0.025638  0.052832
33     2.179844        1      26  0.020561  0.006582
28     2.117775        1      27  0.017354 -0.035027
2      2.085083        1      28  0.022288  0.053926
22     1.949974        1      29  0.024870  0.053917
38     1.891319        1      30  0.023894  0.036616
15     1.820848        1      31  0.018090 -0.027309
37     1.797624        1      32  0.026256  0.020925
31     1.781301        1      33  0.025740  0.059068
12     1.777250        1      34  0.025066  0.055737
14     1.648519        1      35  0.006875  0.032881
29     1.617704        1      36  0.024857  0.016139
5      1.592257        1      37  0.021608  0.020910
10     1.583840        1      38  0.000990  0.013887
11     1.512102        1      39 -0.045943  0.064273
3      1.217884        1      40 -0.100817  0.064692, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
1831  Default  102.000000        request  102.000000  30.0000  30.0000
3328  Default   98.000000          datum   98.000000  29.0000  29.0000
2318  Default  120.000000         client  120.000000  28.0000  28.0000
48    Default   95.000000          invit   95.000000  27.0000  27.0000
3095  Default   88.000000           node   88.000000  26.0000  26.0000
1267  Default   70.000000           list   70.000000  25.0000  25.0000
2386  Default  111.000000         server  111.000000  24.0000  24.0000
2258  Default   89.000000           head   89.000000  23.0000  23.0000
542   Default   73.000000         tunnel   73.000000  22.0000  22.0000
1121  Default   65.000000         servic   65.000000  21.0000  21.0000
1383  Default   59.000000           path   59.000000  20.0000  20.0000
1757  Default   69.000000        control   69.000000  19.0000  19.0000
799   Default   61.000000          frame   61.000000  18.0000  18.0000
2589  Default   59.000000          would   59.000000  17.0000  17.0000
2464  Default   63.000000        segment   63.000000  16.0000  16.0000
443   Default   60.000000         option   60.000000  15.0000  15.0000
457   Default   63.000000          proxi   63.000000  14.0000  14.0000
3526  Default   60.000000           valu   60.000000  13.0000  13.0000
3452  Default   59.000000           link   59.000000  12.0000  12.0000
2010  Default   57.000000        support   57.000000  11.0000  11.0000
1450  Default   72.000000         messag   72.000000  10.0000  10.0000
2685  Default   53.000000         respon   53.000000   9.0000   9.0000
1421  Default   70.000000  